In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans 
from sklearn import preprocessing, svm, model_selection

In [ ]:
path = "C:/Users/ozgur/Downloads/fantasy-basketball-master/data/Boxscores/2018-19"
all_files = glob.glob(path + "/*.csv")

In [ ]:
temp_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    temp_list.append(df)

dataframe = pd.concat(temp_list, axis=0, ignore_index=True)

In [ ]:
print(dataframe)

In [ ]:
missing_data = dataframe.isnull()

for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")


In [ ]:
for index, row in dataframe.iterrows():
    if row['3PA'] is 0 and row.isnull()['3P_perc']:
        dataframe.loc[index, ['3P_perc']] = 0

In [ ]:
dataframe.fillna(0, inplace = True)
dataframe.to_csv("merged.csv", encoding='utf-8', index=False)

In [ ]:
data = np.load('C:\\Users\\ozgur\\Downloads\\fantasy-basketball-master\\data\\Names\\standard_names.npy',allow_pickle=True)
print(data)

In [3]:
merged_path = "C:/Users/ozgur/Desktop/dev/Fantasy-Basketball-Player-Recomendation-System/ML/data/Merged/df_2018-19.csv"
merged_df = pd.read_csv(merged_path, index_col=None, header=0)
# merged_df.drop(['Pos', 'Date', 'Salary', 'FPTS', 'Value', 'PG', 'SG', 'F', 'C', 'Name', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA'], axis=1, inplace=True)

for index, row in merged_df.iterrows():
    if row['W'] == row['Team']:
        merged_df.loc[index, ['W']] = 1
    else:
        merged_df.loc[index, ['W']] = 0

# merged_df.drop(['Team'], axis=1, inplace=True)
merged_df.drop(['L'], axis=1, inplace=True)
# merged_df.to_csv("merged.csv", encoding='utf-8', index=False)
print(merged_df)

                  Name    Pos      Date  Salary  Starter Team   FPTS  \
0         Terry Rozier  PG/SG  20181016  4500.0        0  BOS  24.50   
1         Kyrie Irving     PG  20181016  7600.0        1  BOS  21.50   
2         Marcus Smart  PG/SG  20181016  4400.0        0  BOS  15.25   
3       Brad Wanamaker     PG  20181016  3000.0        0  BOS   3.25   
4         Jayson Tatum  SF/PF  20181016  5600.0        1  BOS  40.75   
...                ...    ...       ...     ...      ...  ...    ...   
28906    Royce O'Neale     SF  20190410  3600.0        1  UTA  10.75   
28907  Tyler Cavanaugh     PF  20190410  3000.0        0  UTA   5.25   
28908        Ekpe Udoh      C  20190410  3300.0        1  UTA  47.25   
28909     Tony Bradley      C  20190410  3000.0        0  UTA  37.50   
28910   Derrick Favors   PF/C  20190410  6200.0        0  UTA   0.00   

           Value  Home  W  ...  DRB_perc  ORB_perc  BLK_perc  TOV_perc  \
0       5.444444     1  1  ...      29.6       0.0       3.0 

In [4]:
merged_df.Date = merged_df.Date.astype(str).str.slice(0, 4) + '/' + merged_df.Date.astype(str).str.slice(4, 6) + '/' + merged_df.Date.astype(str).str.slice(6, 8)
merged_df.Date = pd.to_datetime(merged_df.Date)
merged_df.to_csv("merged.csv", encoding='utf-8', index=False)

In [5]:
days = (merged_df.loc[11000, ['Date']] - merged_df.loc[56, ['Date']]).dt.days
print(merged_df.loc[11000, ['Date']], merged_df.loc[56, ['Date']], days)

Date    2018-12-19 00:00:00
Name: 11000, dtype: object Date    2018-10-17 00:00:00
Name: 56, dtype: object Date    63
dtype: int64


In [14]:
merged_path = "C:/Users/ozgur/Desktop/dev/Fantasy-Basketball-Player-Recomendation-System/ML/main_merged5v2.csv"
merged_df = pd.read_csv(merged_path, index_col=None, header=0)
merged_df.columns

Index(['Name', 'Pos', 'Date', 'Starter', 'Team', 'Home', 'W', 'MP', 'FG',
       'FGA', 'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA', 'FT_perc', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'USG_perc', 'DRtg', 'ORtg',
       'AST_perc', 'DRB_perc', 'ORB_perc', 'BLK_perc', 'TOV_perc', 'STL_perc',
       'eFG_perc', 'PG', 'SG', 'F', 'C', 'PTS_AVG Last Month',
       'PTS_AVG Last Week', 'AST Last Week', 'PTS_AVG', 'AST Last Month',
       'MP_AVG', 'FG_AVG', 'FGA_AVG', 'FG_perc_AVG', 'FG_AVG Last Week',
       'FGA_AVG Last Week', 'FG_perc_AVG Last Week', 'AST_AVG',
       'FG_AVG Last Month', 'FGA_AVG Last Month', 'FG_perc_AVG Last Month',
       'Opp', 'BLK_AVG', 'BLK Last Week', 'BLK Last Month', 'TRB_AVG',
       'STL_AVG', 'MP Last Month', 'MP Last Week', 'ORtg_AVG', 'eFG_perc_AVG',
       '3PA_AVG', 'TOV_AVG', 'DRtg_AVG', 'USG_perc_AVG', '3P_AVG',
       'ORtg Last Month'],
      dtype='object')

In [15]:
WEEK = 7
MONTH = 30 

merged_df['ORtg Last Month'] = np.nan
merged_df.Date = pd.to_datetime(merged_df.Date)
temp_df = merged_df.drop(['Pos', 'Starter', 'Team', 'Home', 'W', 'MP', 'FG',
       'FGA', 'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA', 'FT_perc', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'USG_perc', 'DRtg',
       'AST_perc', 'DRB_perc', 'ORB_perc', 'BLK_perc', 'TOV_perc', 'STL_perc',
       'eFG_perc', 'PG', 'SG', 'F', 'C', 'PTS_AVG Last Month',
       'PTS_AVG Last Week', 'AST Last Week', 'PTS_AVG', 'AST Last Month',
       'MP_AVG', 'FG_AVG', 'FGA_AVG', 'FG_perc_AVG', 'FG_AVG Last Week',
       'FGA_AVG Last Week', 'FG_perc_AVG Last Week', 'AST_AVG',
       'FG_AVG Last Month', 'FGA_AVG Last Month', 'FG_perc_AVG Last Month',
       'Opp', 'BLK_AVG', 'BLK Last Week', 'BLK Last Month', 'TRB_AVG',
       'STL_AVG', 'MP Last Month', 'MP Last Week', 'ORtg_AVG', 'eFG_perc_AVG',
       '3PA_AVG', 'TOV_AVG', 'DRtg_AVG', 'USG_perc_AVG', '3P_AVG',
       'ORtg Last Month'], axis=1)

for index_i, row_i in temp_df.iterrows():
    count = 0
    total = 0

    for index_j, row_j in temp_df.iterrows():
        if index_j >= index_i:
            break
        elif (row_i['Name'] == row_j['Name']) and ((row_i['Date'] - row_j['Date']).days <= MONTH):  
            count += 1
            total += row_j['ORtg']
     
    if total is 0:
        merged_df.loc[index_i, ['ORtg Last Month']] = 0
    else:
        merged_df.loc[index_i, ['ORtg Last Month']] = total/count
    
    print(index_i, total, count, merged_df.loc[index_i, ['ORtg Last Month']])
    
print(merged_df)
# and (row_i['Date'] - row_j['Date']).days <= MONTH:

0 0 0 ORtg Last Month    0
Name: 0, dtype: object
1 0 0 ORtg Last Month    0
Name: 1, dtype: object
2 0 0 ORtg Last Month    0
Name: 2, dtype: object
3 0 0 ORtg Last Month    0
Name: 3, dtype: object
4 0 0 ORtg Last Month    0
Name: 4, dtype: object
5 0 0 ORtg Last Month    0
Name: 5, dtype: object
6 0 0 ORtg Last Month    0
Name: 6, dtype: object
7 0 0 ORtg Last Month    0
Name: 7, dtype: object
8 0 0 ORtg Last Month    0
Name: 8, dtype: object
9 0 0 ORtg Last Month    0
Name: 9, dtype: object
10 0 0 ORtg Last Month    0
Name: 10, dtype: object
11 0 0 ORtg Last Month    0
Name: 11, dtype: object
12 0 0 ORtg Last Month    0
Name: 12, dtype: object
13 0 0 ORtg Last Month    0
Name: 13, dtype: object
14 0 0 ORtg Last Month    0
Name: 14, dtype: object
15 0 0 ORtg Last Month    0
Name: 15, dtype: object
16 0 0 ORtg Last Month    0
Name: 16, dtype: object
17 0 0 ORtg Last Month    0
Name: 17, dtype: object
18 0 0 ORtg Last Month    0
Name: 18, dtype: object
19 0 0 ORtg Last Month    0
Name

156 0 0 ORtg Last Month    0
Name: 156, dtype: object
157 0 0 ORtg Last Month    0
Name: 157, dtype: object
158 0 0 ORtg Last Month    0
Name: 158, dtype: object
159 0 0 ORtg Last Month    0
Name: 159, dtype: object
160 0 0 ORtg Last Month    0
Name: 160, dtype: object
161 0 0 ORtg Last Month    0
Name: 161, dtype: object
162 0 0 ORtg Last Month    0
Name: 162, dtype: object
163 0 0 ORtg Last Month    0
Name: 163, dtype: object
164 0 0 ORtg Last Month    0
Name: 164, dtype: object
165 0 0 ORtg Last Month    0
Name: 165, dtype: object
166 0 0 ORtg Last Month    0
Name: 166, dtype: object
167 0 0 ORtg Last Month    0
Name: 167, dtype: object
168 0 0 ORtg Last Month    0
Name: 168, dtype: object
169 0 0 ORtg Last Month    0
Name: 169, dtype: object
170 0 0 ORtg Last Month    0
Name: 170, dtype: object
171 0 0 ORtg Last Month    0
Name: 171, dtype: object
172 0 0 ORtg Last Month    0
Name: 172, dtype: object
173 0 0 ORtg Last Month    0
Name: 173, dtype: object
174 0 0 ORtg Last Month    0

310 0 0 ORtg Last Month    0
Name: 310, dtype: object
311 0 0 ORtg Last Month    0
Name: 311, dtype: object
312 0 0 ORtg Last Month    0
Name: 312, dtype: object
313 0 0 ORtg Last Month    0
Name: 313, dtype: object
314 0 0 ORtg Last Month    0
Name: 314, dtype: object
315 0 0 ORtg Last Month    0
Name: 315, dtype: object
316 0 0 ORtg Last Month    0
Name: 316, dtype: object
317 0 0 ORtg Last Month    0
Name: 317, dtype: object
318 0 0 ORtg Last Month    0
Name: 318, dtype: object
319 0 0 ORtg Last Month    0
Name: 319, dtype: object
320 0 0 ORtg Last Month    0
Name: 320, dtype: object
321 0 0 ORtg Last Month    0
Name: 321, dtype: object
322 0 0 ORtg Last Month    0
Name: 322, dtype: object
323 0 0 ORtg Last Month    0
Name: 323, dtype: object
324 0 0 ORtg Last Month    0
Name: 324, dtype: object
325 0 0 ORtg Last Month    0
Name: 325, dtype: object
326 0 0 ORtg Last Month    0
Name: 326, dtype: object
327 0 0 ORtg Last Month    0
Name: 327, dtype: object
328 0 0 ORtg Last Month    0

461 116 1 ORtg Last Month    116
Name: 461, dtype: object
462 0 1 ORtg Last Month    0
Name: 462, dtype: object
463 0 1 ORtg Last Month    0
Name: 463, dtype: object
464 220 1 ORtg Last Month    220
Name: 464, dtype: object
465 42 1 ORtg Last Month    42
Name: 465, dtype: object
466 130 1 ORtg Last Month    130
Name: 466, dtype: object
467 117 1 ORtg Last Month    117
Name: 467, dtype: object
468 121 1 ORtg Last Month    121
Name: 468, dtype: object
469 46 1 ORtg Last Month    46
Name: 469, dtype: object
470 127 1 ORtg Last Month    127
Name: 470, dtype: object
471 232 1 ORtg Last Month    232
Name: 471, dtype: object
472 111 1 ORtg Last Month    111
Name: 472, dtype: object
473 129 1 ORtg Last Month    129
Name: 473, dtype: object
474 136 1 ORtg Last Month    136
Name: 474, dtype: object
475 111 1 ORtg Last Month    111
Name: 475, dtype: object
476 132 1 ORtg Last Month    132
Name: 476, dtype: object
477 0 0 ORtg Last Month    0
Name: 477, dtype: object
478 112 1 ORtg Last Month    1

608 154 1 ORtg Last Month    154
Name: 608, dtype: object
609 0 1 ORtg Last Month    0
Name: 609, dtype: object
610 128 1 ORtg Last Month    128
Name: 610, dtype: object
611 151 1 ORtg Last Month    151
Name: 611, dtype: object
612 0 1 ORtg Last Month    0
Name: 612, dtype: object
613 0 1 ORtg Last Month    0
Name: 613, dtype: object
614 149 1 ORtg Last Month    149
Name: 614, dtype: object
615 132 1 ORtg Last Month    132
Name: 615, dtype: object
616 0 1 ORtg Last Month    0
Name: 616, dtype: object
617 180 2 ORtg Last Month    90
Name: 617, dtype: object
618 197 2 ORtg Last Month    98.5
Name: 618, dtype: object
619 236 2 ORtg Last Month    118
Name: 619, dtype: object
620 200 2 ORtg Last Month    100
Name: 620, dtype: object
621 202 2 ORtg Last Month    101
Name: 621, dtype: object
622 210 2 ORtg Last Month    105
Name: 622, dtype: object
623 195 2 ORtg Last Month    97.5
Name: 623, dtype: object
624 0 2 ORtg Last Month    0
Name: 624, dtype: object
625 0 2 ORtg Last Month    0
Name

...

28297 1071 13 ORtg Last Month    82.3846
Name: 28297, dtype: object
28298 748 7 ORtg Last Month    106.857
Name: 28298, dtype: object
28299 1535 13 ORtg Last Month    118.077
Name: 28299, dtype: object
28300 1363 13 ORtg Last Month    104.846
Name: 28300, dtype: object
28301 1058 13 ORtg Last Month    81.3846
Name: 28301, dtype: object
28302 813 11 ORtg Last Month    73.9091
Name: 28302, dtype: object
28303 1389 14 ORtg Last Month    99.2143
Name: 28303, dtype: object
28304 1291 14 ORtg Last Month    92.2143
Name: 28304, dtype: object
28305 1115 14 ORtg Last Month    79.6429
Name: 28305, dtype: object
28306 179 2 ORtg Last Month    89.5
Name: 28306, dtype: object
28307 788 12 ORtg Last Month    65.6667
Name: 28307, dtype: object
28308 1974 14 ORtg Last Month    141
Name: 28308, dtype: object
28309 673 12 ORtg Last Month    56.0833
Name: 28309, dtype: object
28310 1107 14 ORtg Last Month    79.0714
Name: 28310, dtype: object
28311 0 0 ORtg Last Month    0
Name: 28311, dtype: object
2831

28420 732 11 ORtg Last Month    66.5455
Name: 28420, dtype: object
28421 1030 13 ORtg Last Month    79.2308
Name: 28421, dtype: object
28422 1283 13 ORtg Last Month    98.6923
Name: 28422, dtype: object
28423 1584 13 ORtg Last Month    121.846
Name: 28423, dtype: object
28424 563 8 ORtg Last Month    70.375
Name: 28424, dtype: object
28425 900 13 ORtg Last Month    69.2308
Name: 28425, dtype: object
28426 438 3 ORtg Last Month    146
Name: 28426, dtype: object
28427 1174 13 ORtg Last Month    90.3077
Name: 28427, dtype: object
28428 345 5 ORtg Last Month    69
Name: 28428, dtype: object
28429 1576 13 ORtg Last Month    121.231
Name: 28429, dtype: object
28430 1513 13 ORtg Last Month    116.385
Name: 28430, dtype: object
28431 1606 14 ORtg Last Month    114.714
Name: 28431, dtype: object
28432 899 14 ORtg Last Month    64.2143
Name: 28432, dtype: object
28433 1137 14 ORtg Last Month    81.2143
Name: 28433, dtype: object
28434 0 3 ORtg Last Month    0
Name: 28434, dtype: object
28435 250

28544 0 7 ORtg Last Month    0
Name: 28544, dtype: object
28545 1801 15 ORtg Last Month    120.067
Name: 28545, dtype: object
28546 1828 15 ORtg Last Month    121.867
Name: 28546, dtype: object
28547 1566 15 ORtg Last Month    104.4
Name: 28547, dtype: object
28548 169 13 ORtg Last Month    13
Name: 28548, dtype: object
28549 1738 14 ORtg Last Month    124.143
Name: 28549, dtype: object
28550 1303 14 ORtg Last Month    93.0714
Name: 28550, dtype: object
28551 81 13 ORtg Last Month    6.23077
Name: 28551, dtype: object
28552 1230 12 ORtg Last Month    102.5
Name: 28552, dtype: object
28553 1611 14 ORtg Last Month    115.071
Name: 28553, dtype: object
28554 1329 14 ORtg Last Month    94.9286
Name: 28554, dtype: object
28555 1364 14 ORtg Last Month    97.4286
Name: 28555, dtype: object
28556 203 10 ORtg Last Month    20.3
Name: 28556, dtype: object
28557 0 4 ORtg Last Month    0
Name: 28557, dtype: object
28558 1474 14 ORtg Last Month    105.286
Name: 28558, dtype: object
28559 1750 14 OR

28668 1056 9 ORtg Last Month    117.333
Name: 28668, dtype: object
28669 1336 13 ORtg Last Month    102.769
Name: 28669, dtype: object
28670 199 11 ORtg Last Month    18.0909
Name: 28670, dtype: object
28671 1353 13 ORtg Last Month    104.077
Name: 28671, dtype: object
28672 1167 13 ORtg Last Month    89.7692
Name: 28672, dtype: object
28673 1321 13 ORtg Last Month    101.615
Name: 28673, dtype: object
28674 1242 11 ORtg Last Month    112.909
Name: 28674, dtype: object
28675 372 8 ORtg Last Month    46.5
Name: 28675, dtype: object
28676 0 1 ORtg Last Month    0
Name: 28676, dtype: object
28677 1702 13 ORtg Last Month    130.923
Name: 28677, dtype: object
28678 403 13 ORtg Last Month    31
Name: 28678, dtype: object
28679 1797 13 ORtg Last Month    138.231
Name: 28679, dtype: object
28680 1820 13 ORtg Last Month    140
Name: 28680, dtype: object
28681 211 2 ORtg Last Month    105.5
Name: 28681, dtype: object
28682 1582 13 ORtg Last Month    121.692
Name: 28682, dtype: object
28683 658 9

28792 1796 15 ORtg Last Month    119.733
Name: 28792, dtype: object
28793 1671 15 ORtg Last Month    111.4
Name: 28793, dtype: object
28794 1860 15 ORtg Last Month    124
Name: 28794, dtype: object
28795 169 13 ORtg Last Month    13
Name: 28795, dtype: object
28796 890 11 ORtg Last Month    80.9091
Name: 28796, dtype: object
28797 1292 14 ORtg Last Month    92.2857
Name: 28797, dtype: object
28798 1324 13 ORtg Last Month    101.846
Name: 28798, dtype: object
28799 1330 13 ORtg Last Month    102.308
Name: 28799, dtype: object
28800 1855 14 ORtg Last Month    132.5
Name: 28800, dtype: object
28801 998 11 ORtg Last Month    90.7273
Name: 28801, dtype: object
28802 795 12 ORtg Last Month    66.25
Name: 28802, dtype: object
28803 1432 14 ORtg Last Month    102.286
Name: 28803, dtype: object
28804 278 9 ORtg Last Month    30.8889
Name: 28804, dtype: object
28805 1180 14 ORtg Last Month    84.2857
Name: 28805, dtype: object
28806 1294 14 ORtg Last Month    92.4286
Name: 28806, dtype: object
2

In [16]:
merged_df.to_csv("main_merged6.csv", encoding='utf-8', index=False)

In [10]:
merged_df.columns

Index(['Name', 'Pos', 'Date', 'Starter', 'Team', 'Home', 'W', 'W_PTS', 'L_PTS',
       'MP', 'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA',
       'FT_perc', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'USG_perc',
       'DRtg', 'ORtg', 'AST_perc', 'DRB_perc', 'ORB_perc', 'BLK_perc',
       'TOV_perc', 'STL_perc', 'eFG_perc', 'PG', 'SG', 'F', 'C',
       'PTS_AVG Last Month', 'PTS_AVG Last Week', 'AST Last Week', 'PTS_AVG',
       'AST Last Month', 'MP_AVG', 'FG_AVG', 'FGA_AVG', 'FG_perc_AVG',
       'FG_AVG Last Week', 'FGA_AVG Last Week', 'FG_perc_AVG Last Week',
       'AST_AVG', 'FG_AVG Last Month', 'FGA_AVG Last Month',
       'FG_perc_AVG Last Month', 'Opp', 'BLK_AVG', 'BLK Last Week',
       'BLK Last Month', 'TRB_AVG', 'STL_AVG', 'MP Last Month',
       'MP Last Week'],
      dtype='object')

In [ ]:
def add_binary_positions(df):
    zeros = [0 for i in range(df.shape[0])]
    PG, SG, SF, PF, C = zeros.copy(), zeros.copy(), zeros.copy(), zeros.copy(), zeros.copy()
        
    for i in range(df.shape[0]):
        if 'PG' in df.loc[i,'Pos']:
            PG[i] = 1
            
        elif 'SG' in df.loc[i,'Pos']:
            SG[i] = 1

        elif 'C' in df.loc[i,'Pos']:
            C[i] = 1
            
        elif 'PF' in df.loc[i,'Pos']:
            C[i] = 1
            
        elif 'SF' in df.loc[i,'Pos']:
            C[i] = 1
            
    df['PG'] = PG
    df['SG'] = SG
    df['SF'] = SF
    df['PF'] = SF
    df['C'] = C

In [92]:
merged_path = "C:/Users/ozgur/Desktop/dev/Fantasy-Basketball-Player-Recomendation-System/ML/merged1.csv"
merged_df = pd.read_csv(merged_path, index_col=None, header=0)
temp_df = merged_df.drop(['Pos', 'Date', 'Name', 'Team', 'Value', 'Home', 'W', 'W_PTS', 'L_PTS', 'Salary', 'FPTS'], axis=1)

k_means = KMeans(init = "k-means++", n_clusters = 8, n_init = 100)
k_means.fit(temp_df)

merged_df['POS Type'] = k_means.labels_
merged_df.to_csv("merged2.csv", encoding='utf-8', index=False)

In [13]:
merged_df.Date[12000:14000].to_list()

[Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-